 **Что я сделала/помогло**:

 1) Использовала предобученную модель resnext50_32x4d из torch

 2) Размер входных изображений приводила к размеру 256, вместо 128

 3) Использовала усреднение моделей, но к сожалению, успела обучить только 2 модели (в сумме охватывали всю трейновую выборку), а в идеале хотела бы 5 моделей (так как размер валидационной выборки 20%), которые обучаются на 80% выборки, но на разных частях, таким образом, чтобы валидационные части  этих моделей не пересекались. 

 4) Каждые 2 эпохи я сохраняла модели и выбирала learning_rate, основываясь на результатах предыдущих

 5) Не забыла, что нужно усреднять не итоговые файлы для отправки моделей, которые уже округлены, а усреднять результаты до округления.

 **Комментарии**:
 
 Не успела дообучить даже вторую модель, так как дедлайн был очень близок 

 Пробовала lr_sheduler, но удобнее показалось вручную выбрать learning_rate, эпох всего было 7 . В итоге получилось, что каждые 2 эпохи я уменьшала learning_rate примерно в 2 раза


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms

from hack_utils import NUM_PTS, CROP_SIZE
from hack_utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from hack_utils import ThousandLandmarksDataset
from hack_utils import restore_landmarks_batch, create_submission_float

In [0]:
np.random.seed(1234)
GPU=True
device = torch.device("cuda: 0") if GPU else torch.device("cpu")

train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
])

In [0]:
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training...", position=0, leave=True):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(train_loss)


def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation...", position=0, leave=True):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)


def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...", position=0, leave=True)):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

# ***Train first model***

In [0]:
# Из всей выборки я беру первые 80% в обучение, остальное в валидационную выборку
train_dataset = ThousandLandmarksDataset(os.path.join('data', 'train'), train_transforms, number_sample = 1, split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=32, num_workers=4, pin_memory=True,
                                    shuffle=True, drop_last=True)
val_dataset = ThousandLandmarksDataset(os.path.join('data', 'train'), train_transforms, number_sample = 1, split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=32, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

393931it [02:01, 3244.06it/s]


In [0]:
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model.to(device)

loss_fn = fnn.mse_loss
best_val_loss = np.inf

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.001, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_0905_2epoch.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss: 8.7e+01 	val loss: 1.1e+01


validation...: 100%|██████████| 2463/2463 [05:37<00:00,  7.31it/s]


Epoch # 1:	train loss: 1.1e+01 	val loss:   9.4


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_0905_4epoch.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   9.0 	val loss:   7.7


validation...: 100%|██████████| 2463/2463 [05:37<00:00,  7.31it/s]


Epoch # 1:	train loss:   7.6 	val loss:   6.9


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.0003, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_0905_6epoch.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   6.6 	val loss:   6.6


validation...: 100%|██████████| 2463/2463 [05:37<00:00,  7.31it/s]


Epoch # 1:	train loss:   6.0 	val loss:   6.3


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.0001, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    #train_loss2 = train(model, train_dataloader_flip, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_0905_8epoch.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   5.1 	val loss:   5.9


validation...: 100%|██████████| 2463/2463 [05:38<00:00,  7.28it/s]

Epoch # 1:	train loss:   4.8 	val loss:   6.1


In [0]:
test_dataset = ThousandLandmarksDataset(os.path.join('data', 'test'), train_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=64, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

with open(f"resnext_crop_0905_8epoch.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"resnext_crop_0905_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                  "landmarks": test_predictions}, fp)

create_submission_float('data', test_predictions, f"resnext_crop_0905_8epoch_float.csv")

41209it [00:00, 412085.48it/s]

99819


99820it [00:00, 414275.00it/s]
test prediction...: 100%|██████████| 1560/1560 [06:07<00:00,  4.25it/s]


# **Train second model**

In [0]:
# Из всей выборки я беру первые 20% в валидацию, остальной в обучающую выборку
train_dataset = ThousandLandmarksDataset(os.path.join('data', 'train'), train_transforms, number_sample = 2, split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=32, num_workers=4, pin_memory=True,
                                    shuffle=True, drop_last=True)
val_dataset = ThousandLandmarksDataset(os.path.join('data', 'train'), train_transforms, number_sample = 2, split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=32, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

393931it [08:37, 761.81it/s]
78731it [03:27, 624.56it/s]

In [0]:
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model.to(device)

loss_fn = fnn.mse_loss
best_val_loss = np.inf

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.001, amsgrad=True)
print("Ready for training...")
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_1105_2epoch_val.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss: 1.9e+01 	val loss: 1e+01


validation...: 100%|██████████| 2463/2463 [05:35<00:00,  7.34it/s]


Epoch # 1:	train loss:   9.5 	val loss:   8.8


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_1105_4epoch_val.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   8.1 	val loss:   7.4


validation...: 100%|██████████| 2463/2463 [05:38<00:00,  7.27it/s]


Epoch # 1:	train loss:   7.0 	val loss:   6.7


In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.0002, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_1105_6epoch_val.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   6.0 	val loss:   6.3


validation...: 100%|██████████| 2463/2463 [05:35<00:00,  7.33it/s]


Epoch # 1:	train loss:   5.6 	val loss:   6.2


In [0]:
# Не успела сделать 8 эпоху, остановилась на 7, так как до конца соревнования оставалось меньше часа
optimizer = optim.Adam(model.parameters(), lr=0.00008, amsgrad=True)
for epoch in range(2):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2} \tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"resnext_crop_1105_7epoch_val.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)

training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Ready for training...


training...:   0%|          | 0/9848 [00:00<?, ?it/s]

Epoch # 0:	train loss:   4.9 	val loss:   6.0


training...:   0%|          | 26/9848 [00:13<1:21:23,  2.01it/s]

KeyboardInterrupt: ignored

In [0]:
test_dataset = ThousandLandmarksDataset(os.path.join('data', 'test'), train_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=64, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

with open(f"resnext_crop_1105_7epoch_val.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"resnext_crop_1105_7epoch_val_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                  "landmarks": test_predictions}, fp)

create_submission_float('data', test_predictions, f"resnext_crop_1105_7epoch_val_float.csv")

33789it [00:00, 337884.68it/s]

99819


99820it [00:00, 394651.04it/s]
test prediction...: 100%|██████████| 1560/1560 [06:00<00:00,  4.33it/s]


In [0]:
test_dataset = ThousandLandmarksDataset(os.path.join('data', 'test'), train_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=64, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)

with open(f"resnext_crop_1105_7epoch_val.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"resnext_crop_1105_7epoch_val_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                  "landmarks": test_predictions}, fp)

create_submission_float('data', test_predictions, f"resnext_crop_1105_7epoch_val_float.csv")

### **Avarage results of 2 models**


In [0]:
result = pd.read_csv("resnext_crop_0905_8epoch_float.csv")
result.head()

,file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y
0,c59b2834147079f3e34627f1e75656b1.jpg,65.289970,59.937138,70.338371,56.332489,55.483498,86.452248,67.929428,61.757137,89.226089,70.613457,40.369778,63.823235,74.606850,60.958389,32.622311,26.975384,65.644073,61.498074,38.141346,61.610332,74.171257,53.091259,54.132458,62.215893,49.997978,91.427773,21.663687,57.555695,65.078163,59.888191,29.570999,100.896927,68.133827,62.289215,71.304230,59.205624,44.271946,62.473759,70.505905,62.140633,74.830254,61.188541,44.121422,62.709545,63.677315,56.418205,62.844940,89.865845,56.452419,89.477341,23.119942,39.836216,54.895264,83.446877,30.470196,102.722084,43.539955,23.845037,49.076118,66.343758
1,85629eb20fdbbb98d0ee20ed9f3ba233.jpg,86.604904,161.852859,90.676949,124.182060,148.935333,105.741631,116.952682,117.508156,190.587509,111.383430,52.386288,95.318085,189.483246,150.817780,65.380463,102.135033,134.338806,119.892616,52.955753,86.626900,133.003601,176.596298,78.147865,117.438576,95.064415,174.635635,93.562759,163.046768,149.334030,222.012527,192.054779,127.764160,120.189980,121.112045,130.291061,115.873230,126.820946,114.740402,109.260620,134.423141,125.571854,172.621872,75.524384,152.389084,93.500008,164.751389,118.751350,171.263397,75.780579,150.764572,59.061543,102.288620,131.076828,173.122162,81.574219,188.150452,56.504631,160.486572,129.315155,172.836670
2,4df0124462ce32e0e8dd5d96a7d6496d.jpg,195.509933,156.747925,123.747940,265.017242,98.402031,177.035858,153.027573,184.670807,137.832581,187.804825,145.510941,324.325714,112.430656,311.059814,225.217010,159.073578,230.654861,187.587128,138.244110,198.143005,76.148117,172.251083,120.910835,261.621796,228.126038,195.723862,138.298462,196.495544,84.713387,166.796127,102.974770,185.850296,165.426193,170.215851,59.291908,204.647354,190.801620,171.766769,231.002167,195.942291,169.505524,167.644485,155.439270,272.911774,178.284485,175.709457,201.887482,163.440704,140.567932,231.564438,187.251114,173.731506,74.798996,249.275528,174.004227,176.258392,151.215240,233.701950,180.787613,166.191696
3,fca085b03b834d9d94170e8f1453276e.jpg,54.858025,124.938347,44.060642,114.658142,83.951370,80.380371,86.224159,68.858101,102.692101,73.330170,70.421272,81.548904,56.357353,116.730759,82.897644,78.962608,37.524006,79.481293,58.982475,86.319122,47.385582,79.418686,85.241997,73.448631,22.218214,83.037544,89.134712,135.665680,94.488983,74.116348,49.378391,95.773445,41.550312,69.295502,58.074215,107.493385,40.149891,73.870163,70.027931,97.688919,75.374588,118.145020,67.697044,151.209763,78.992340,68.858498,73.626793,78.942162,52.626411,86.995766,57.421654,120.473328,38.625847,69.170914,99.382393,112.343788,61.048882,151.999115,40.006672,78.419174
4,67b598c8621b1f7135dfbc1c47fe2893.jpg,86.050453,74.927452,86.869339,29.431252,77.035370,77.407791,104.914093,64.660080,92.864616,74.369797,88.870346,59.549259,72.398514,114.471344,59.658508,29.668930,119.427193,68.702644,59.509800,74.914352,83.689598,118.637299,115.985405,52.626774,67.507240,77.204056,43.232769,37.037220,117.198418,56.544952,65.490097,29.141897,42.193939,119.074944,77.423073,28.745342,62.993103,74.706917,89.776695,59.429005,49.880432,71.645538,89.542793,29.899483,50.674740,71.284691,72.574974,120.107437,97.441528,73.512093,95.814941,72.930016,74.856903,150.038879,100.982933,60.269016,100.760208,1

In [0]:
result2 = pd.read_csv("resnext_crop_1105_7epoch_val_float.csv")
result2.head()

,file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y
0,c59b2834147079f3e34627f1e75656b1.jpg,65.392578,59.615261,70.190727,56.488220,55.572823,85.849319,67.991051,61.433952,88.580704,70.644417,40.349747,63.666046,74.662483,60.698917,32.867229,27.755688,65.737633,61.170624,38.044769,61.509892,73.934937,53.109287,54.222141,62.292236,50.123505,91.012070,21.738428,57.810612,65.145142,59.638927,29.824150,100.931595,68.160378,62.008144,71.345795,58.946754,44.181683,62.171413,70.569687,61.850960,74.956856,60.922779,44.081459,62.428162,63.659145,56.926640,63.212200,89.313080,56.783726,88.784615,23.386187,40.406445,54.938709,83.009239,30.690758,102.708832,43.599663,24.621426,49.164383,65.941910
1,85629eb20fdbbb98d0ee20ed9f3ba233.jpg,86.936821,161.646240,90.674019,123.855995,149.018631,105.175064,116.970352,117.087135,188.071472,111.608963,51.825832,95.129921,188.475052,151.942108,66.331589,101.555252,134.143234,119.832169,52.408600,86.377457,133.566620,176.539948,78.611450,116.897896,95.772980,174.619186,93.614349,162.982239,148.848663,222.524918,189.404984,127.778831,120.009567,120.818748,130.111542,115.896629,126.749939,114.616875,108.911369,133.889938,126.089989,173.036209,76.440697,151.774551,93.375710,164.557037,118.986366,172.091400,76.706772,150.179703,60.289459,101.267944,131.655212,173.319305,81.985497,185.949188,57.724846,159.577042,129.843903,173.127487
2,4df0124462ce32e0e8dd5d96a7d6496d.jpg,196.973755,155.756607,123.359406,265.357483,99.967987,177.063889,153.933060,185.480957,138.533218,189.347870,150.112564,324.716492,116.373810,312.737274,225.506790,159.925827,233.283447,189.307953,138.780945,199.491577,75.648857,173.101547,120.145607,262.728668,228.857712,196.515182,138.730331,197.708450,84.375221,166.899689,103.855927,185.003891,166.472839,170.117310,60.365166,208.314453,191.633469,171.543213,233.819504,197.578369,169.784454,167.460693,155.116943,271.180634,178.686096,174.827805,203.758087,163.181183,140.594650,232.140182,187.835266,173.226273,75.679543,252.504684,174.578323,175.328461,151.395920,234.605225,181.373795,165.174423
3,fca085b03b834d9d94170e8f1453276e.jpg,54.758404,124.866936,43.794369,114.156143,84.141609,80.124786,85.870811,69.046097,102.413712,73.498962,70.674858,81.277267,56.409439,116.566811,83.040108,78.837257,37.894554,79.598068,59.186939,86.337372,47.625168,79.446831,85.017174,73.535141,21.880161,83.192123,89.175621,135.470367,94.201645,73.981583,49.385807,95.374771,41.684578,69.528763,58.312466,107.266693,40.281124,73.925957,70.472450,97.072754,75.760033,117.579727,68.135956,151.582520,78.740578,69.298355,73.927826,78.777573,52.791183,86.636612,57.440887,120.343063,38.775986,69.329178,99.330009,112.353134,61.508663,152.616333,40.317825,78.533997
4,67b598c8621b1f7135dfbc1c47fe2893.jpg,85.234634,74.825966,86.675003,27.886557,76.805016,76.943123,102.919914,63.845997,91.863091,74.284286,87.006554,60.016533,72.599251,114.055031,59.448631,28.250561,118.875572,67.559212,59.655693,74.651115,83.722244,118.498871,115.551292,51.472351,67.288338,77.206367,42.989067,35.914619,116.761566,55.367825,65.285843,27.659960,42.888515,118.700134,77.196335,27.191212,62.933899,74.147705,87.869331,59.784710,50.444717,71.885208,89.397888,28.331871,51.113289,71.569786,72.843948,120.068047,96.451881,73.408684,94.747818,72.892586,74.937187,150.218079,98.944626,59.703568,101.130173,13

In [0]:
# усредняем
result_union = pd.DataFrame(result['file_name'].copy())
for column in result.columns:
    if column!='file_name':
        result_union[column]= result[column]/2 + result2[column]/2
# округляем
result_union = result_union.round(0)
for column in result_union.columns:
    if column!='file_name':
        result_union[column]=result_union[column].astype(int) 
    
result_union.to_csv('union_2_NN_float_baseline.csv',index=False)   